### Functional Approach

In [113]:
import boto3
import pandas as pd
# To save the file as parquet format in S3, we use BytesIO
from io import StringIO, BytesIO
from datetime import datetime, timedelta

In [114]:
# Adapter Layer

def read_csv_to_df(bucket, key, decoding = 'utf-8', sep = ','):
    csv_obj = bucket.Object(key=key).get().get('Body').read().decode(decoding)
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=sep)
    return df

def write_df_to_s3(bucket, df, key):
    out_buffer = BytesIO()
    df.to_parquet(out_buffer, index=False)
    bucket.put_object(Body=out_buffer.getvalue(), Key=key)
    return True

def write_df_to_s3_csv(bucket, df, key):
    out_buffer = StringIO()
    df.to_csv(out_buffer, index=False)
    bucket.put_object(Body=out_buffer.getvalue(), Key=key)
    return True

def list_files_in_prefix(bucket, prefix):
    files = [obj.key for obj in bucket.objects.filter(Prefix=prefix)]
    return files

    #objects = [obj.key for obj in bucket.objects.all() if datetime.strptime(obj.key.split('/')[0], src_format).date() >= min_date
    #      and datetime.strptime(obj.key.split('/')[0], src_format).date() <= datetime.strptime(arg_date, src_format).date()]
    #return objects

In [115]:
# Application Layer

def extract(bucket, date_list):
    files = [key for date in date_list for key in list_files_in_prefix(bucket, date)]
    df = pd.concat([read_csv_to_df(bucket, obj) for obj in files], ignore_index=True)
    return df

def transform_report1(df, columns, arg_date):
    df = df.loc[:, columns]
    df.dropna(inplace=True)
    df['opening_price'] = df.sort_values(by = ['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('first')
    df['closing_price'] = df.sort_values(by = ['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('last')
    df = df.groupby(['ISIN','Date'], as_index = False).agg(opening_price_eur=('opening_price', 'min'), 
                                                               closing_price_eur=('closing_price','min'), 
                                                               minimum_price_eur=('MinPrice','min'), 
                                                               maximum_price_eur=('MaxPrice','max'), 
                                                               daily_traded_volume=('TradedVolume','sum'))
    df['prev_closing_price'] = df.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)
    df['change_prev_closing_%'] = (df['closing_price_eur'] - df['prev_closing_price']) / df['prev_closing_price'] * 100
    df.drop(columns=['prev_closing_price'], inplace=True)
    df = df.round(decimals=2)
    df = df[df.Date >= arg_date]
    return df

def load(bucket, df, tgt_key, tgt_format, meta_key, extract_date_list):
    key = tgt_key + datetime.today().strftime("%Y%m%d_%H%M%S") + tgt_format
    write_df_to_s3(bucket, df, key)
    update_meta_file(bucket, meta_key, extract_date_list)
    return True

def etl_report1(src_bucket, tgt_bucket, date_list, columns, arg_date, tgt_key, tgt_format, meta_key):
    df = extract(src_bucket, date_list)
    df = transform_report1(df, columns, arg_date)
    extract_date_list = [date for date in date_list if date >= arg_date]
    load(tgt_bucket, df, tgt_key, tgt_format, meta_key, extract_date_list)
    return True

In [116]:
# Application Layer - not core

def return_date_list(bucket, arg_date, src_format, meta_key):
    min_date = datetime.strptime(arg_date, src_format).date() - timedelta(days=1)
    today = datetime.today().date()
    
    try:
        
        df_meta = read_csv_to_df(bucket, meta_key)
        dates = [(min_date + timedelta(days=x)) for x in range(0, (today-min_date).days + 1)]
        src_dates = set(pd.to_datetime(df_meta['source_date']).dt.date)
        dates_missing = set(dates[1:]) - src_dates

        if dates_missing:
            min_date = min(set(dates[1:]) - src_dates) - timedelta(days=1)
            return_dates = [date.strftime(src_format) for date in dates if date >= min_date]
            return_min_date = (min_date + timedelta(days=1)).strftime(src_format)
        else:
            return_dates = []
            return_min_date = datetime(2200, 1, 1).date()
    
    except bucket.session.client('s3').exceptions.NoSuchKey:
        return_dates = [(min_date + timedelta(days=x)).strftime(src_format) for x in range(0, (today-min_date).days)]
        return_min_date = arg_date
        
    return return_min_date, return_dates

# Function to update metafile in S3 
def update_meta_file(bucket, meta_key, extract_date_list):
    df_new = pd.DataFrame(columns=['source_date', 'datetime_of_processing'])
    df_new['source_date'] = extract_date_list
    df_new['datetime_of_processing'] = datetime.today().strftime('%Y-%m-%d')
    df_old = read_csv_to_df(bucket, meta_key)
    df_all = pd.concat([df_old, df_new])
    write_df_to_s3_csv(bucket, df_all, meta_key)

In [117]:
# Main Function Entrypoint

def main():
    #Parameters/configurations
    #Later read config
    arg_date = '2022-05-06'
    src_format = '%Y-%m-%d'
    src_bucket = 'deutsche-boerse-xetra-pds'
    tgt_bucket = 'xetra-quickanddirtysolution1234'
    columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice', 'EndPrice', 'TradedVolume']
    tgt_key = 'xetra_daily_report_'
    tgt_format = '.parquet'
    meta_key = 'meta_file.csv'
    
    #Init
    s3 = boto3.resource('s3')
    bucket_src = s3.Bucket(src_bucket)
    bucket_tgt = s3.Bucket(tgt_bucket)
    
    # Run Application
    extract_date, date_list = return_date_list(bucket_tgt, arg_date, src_format, meta_key)
    print(f'Extract date: {extract_date}')
    print()
    # Here date_list is the list from arg date to current date and will update meta file in s3 depicting that all the files till today are processed
    print(f'Date List: {date_list}')
    print()
    etl_report1(bucket_src, bucket_tgt, date_list, columns, extract_date, tgt_key, tgt_format, meta_key)

In [118]:
# Execute
main()
print('ETL job executed successfully')

Extract date: 2022-05-06

Date List: ['2022-05-05', '2022-05-06', '2022-05-07', '2022-05-08', '2022-05-09', '2022-05-10', '2022-05-11', '2022-05-12', '2022-05-13', '2022-05-14', '2022-05-15', '2022-05-16', '2022-05-17', '2022-05-18', '2022-05-19', '2022-05-20', '2022-05-21', '2022-05-22', '2022-05-23', '2022-05-24', '2022-05-25', '2022-05-26', '2022-05-27', '2022-05-28', '2022-05-29', '2022-05-30', '2022-05-31', '2022-06-01', '2022-06-02', '2022-06-03', '2022-06-04', '2022-06-05', '2022-06-06', '2022-06-07', '2022-06-08', '2022-06-09', '2022-06-10', '2022-06-11', '2022-06-12', '2022-06-13', '2022-06-14', '2022-06-15']

ETL job executed successfully


### Write to S3 Bucket

### Reading the uploaded file

In [119]:
tgt_bucket = 'xetra-quickanddirtysolution1234'
s3 = boto3.resource('s3')
bucket_tgt = s3.Bucket(tgt_bucket)
for obj in bucket_tgt.objects.all():
    print(obj.key)

meta_file.csv
xetra_daily_report_20220613_160951.parquet
xetra_daily_report_20220614_110223.parquet
xetra_daily_report_20220614_114032.parquet
xetra_daily_report_20220614_124003.parquet
xetra_daily_report_20220615_105344.parquet
xetra_daily_report_20220615_110322.parquet
xetra_daily_report_20220615_114927.parquet
xetra_daily_report_20220615_115114.parquet
xetra_daily_report_20220615_115228.parquet
xetra_daily_report_20220615_115331.parquet
xetra_daily_report_20220615_115504.parquet
xetra_daily_report_20220615_115600.parquet
xetra_daily_report_20220615_115725.parquet
xetra_daily_report_20220615_120045.parquet
xetra_daily_report_20220615_121615.parquet


In [121]:
par_obj = bucket_tgt.Object(key='xetra_daily_report_20220615_121615.parquet').get().get('Body').read()
data = BytesIO(par_obj)
df_report = pd.read_parquet(data)
df_report

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,change_prev_closing_%
0,AT000000STR1,2022-05-06,36.30,36.15,36.15,36.50,350,-3.08
1,AT000000STR1,2022-05-09,36.15,35.30,35.30,36.15,125,-2.35
2,AT000000STR1,2022-05-10,35.80,35.25,35.25,35.90,310,-0.14
3,AT000000STR1,2022-05-11,36.55,36.30,35.90,36.55,540,2.98
4,AT000000STR1,2022-05-12,36.30,35.80,35.80,36.30,295,-1.38
...,...,...,...,...,...,...,...,...
19873,XS2437455608,2022-05-09,26.45,29.24,26.45,29.24,0,10.84
19874,XS2437455608,2022-05-10,28.32,29.32,28.32,29.32,0,0.27
19875,XS2437455608,2022-05-11,28.54,27.39,27.39,28.54,0,-6.60
19876,XS2437455608,2022-05-12,28.86,27.77,27.77,28.93,0,1.39
